# 作業目標: 使用python正規表達式對資料進行清洗處理

這份作業我們會使用詐欺郵件的文本資料來作為清洗與處理的操作。
[資料集](https://www.kaggle.com/rtatman/fraudulent-email-corpus/data#)

### 讀入資料文本
因原始文本較大，先使用部份擷取的**sample_emails.txt**來進行練習

In [1]:
#讀取文本資料

with open("sample_emails.txt", "r", encoding = "utf-8") as file:
    sample_corpus = file.read()

In [ ]:
sample_corpus

### 讀取寄件者資訊
觀察文本資料可以發現, 寄件者資訊都符合以下格式

`From: <收件者姓名> <收件者電子郵件>`

In [8]:
import re
match = re.findall(r"From:.*", sample_corpus)

In [9]:
match

['From: "MR. JAMES NGOLA." <james_ngola2002@maktoob.com>',
 'From: "Mr. Ben Suleman" <bensul2004nng@spinfinder.com>',
 'From: "PRINCE OBONG ELEME" <obong_715@epatra.com>']

### 只讀取寄件者姓名

In [11]:

for s in match:
    x = re.search(r'".*"',s)
    print(x.group())


"MR. JAMES NGOLA."
"Mr. Ben Suleman"
"PRINCE OBONG ELEME"


### 只讀取寄件者電子信箱

In [12]:
for s in match:
    x = re.search(r'(?<=<).*[^>]',s)
    print(x.group())

james_ngola2002@maktoob.com
bensul2004nng@spinfinder.com
obong_715@epatra.com


### 只讀取電子信箱中的寄件機構資訊
ex: james_ngola2002@maktoob.com --> 取maktoob

In [21]:
for s in match:
    x = re.search(r'(?<=@).*(?=.com)', s)
    print(x.group())

maktoob
spinfinder
epatra


### 結合上面的配對方式, 將寄件者的帳號與機構訊返回
ex: james_ngola2002@maktoob.com --> [james_ngola2002, maktoob]

In [23]:
for s in match:
    x = re.search(r'(?<=<).*(?=@)', s)
    y = re.search(r'(?<=@).*(?=.com)', s)
    print(x.group(),",", y.group())

james_ngola2002 , maktoob
bensul2004nng , spinfinder
obong_715 , epatra


### 使用正規表達式對email資料進行處理
這裡我們會使用到python其他的套件協助處理(ex: pandas, email, etc)，這裡我們只需要專注在正規表達式上即可，其他的套件是方便我們整理與處理資料。

### 讀取與切分Email
讀入的email為一個長字串，利用正規表達式切割讀入的資料成一封一封的email，並將結果以list表示。

輸出: [email_1, email_2, email_3, ....]

In [31]:
import re
import pandas as pd
import email
with open("all_emails.txt", "r", encoding = "utf-8", errors='ignore') as file:
    emails = file.read()

emails = re.split(r"From r", emails,  flags = re.M)
emails = emails[1:]
len(emails)

3977

### 從文本中擷取所有寄件者與收件者的姓名和地址

In [33]:
emails_list = [] #創建空list來儲存所有email資訊

for mail in emails[:20]: #只取前20筆資料 (處理速度比較快)
    emails_dict = dict() #創建空字典儲存資訊
    ###取的寄件者姓名與地址###
    
    #Step1: 取的寄件者資訊 (hint: From:)
    sender = re.search(r"From:.*", mail)
    
    #Step2: 取的姓名與地址 (hint: 要注意有時會有沒取到配對的情況)
    if sender is not None:
        sender_mail = re.search(r"\w\S*@.*\b", sender.group())
        sender_name = re.search(r"(?<=\").*(?=\")", sender.group())
    else:
        sender_mail = None
        sender_name = None
    
    #Step3: 將取得的姓名與地址存入字典中
    if sender_mail is not None:
        emails_dict["sender_email"] = sender_mail.group()
    else:
        emails_dict["sender_email"] = sender_mail #None
    
    if sender_name is not None:
        emails_dict["sender_name"] = sender_name.group()
    else:
        emails_dict["sender_name"] = sender_name #None
        
    
    ###取的收件者姓名與地址###
    #Step1: 取的寄件者資訊 (hint: To:)
    receive = re.search(r"To:.*", mail)
    
    #Step2: 取的姓名與地址 (hint: 要注意有時會有沒取到配對的情況)
    if receive is not None:
        r_email = re.search(r"\w\S*@.*\b", receive.group())
        r_name = re.search(r"(?<=\").*(?=\")", receive.group())
    else:
        r_email = None
        r_name = None
        
    #Step3: 將取得的姓名與地址存入字典中
    if r_email is not None:
        emails_dict["receive_email"] = r_email.group()
    else:
        emails_dict["receive_email"] = r_email #None
    
    if r_name is not None:
        emails_dict["receive_name"] = r_name.group()
    else:
        emails_dict["receive_name"] = r_name #None
        
        
    ###取得信件日期###
    #Step1: 取得日期資訊 (hint: To:)
    date_info = re.search(r"Date:.*", mail)
    
    #Step2: 取得詳細日期(只需取得DD MMM YYYY)
    if date_info is not None:
        date = re.search(r"\d+\s\w+\s\d+", date_info.group())
    else:
        date = None
        
    #Step3: 將取得的日期資訊存入字典中
    if date is not None:
        emails_dict["date_sent"] = date.group()
    else:
        emails_dict["date_sent"] = date
        
        
    ###取得信件主旨###
    #Step1: 取得主旨資訊 (hint: Subject:)
    subject_info = re.search(r"Subject: .*", mail)
    
    #Step2: 移除不必要文字 (hint: Subject: )
    if subject_info is not None:
        subject = re.sub(r"Subject: ", "", subject_info.group())
    else:
        subject = None    
    #Step3: 將取得的主旨存入字典中
    emails_dict["subject"] = subject

    
    
    ###取得信件內文###
    #這裡我們使用email package來取出email內文 (可以不需深究，本章節重點在正規表達式)
    try:
        full_email = email.message_from_string(mail)
        body = full_email.get_payload()
        emails_dict["email_body"] = body
    except:
        emails_dict["email_body"] = None
    
    ###將字典加入list###
    emails_list.append(emails_dict)

In [34]:
#將處理結果轉化為dataframe
emails_df = pd.DataFrame(emails_list)
emails_df

,sender_email,sender_name,receive_email,receive_name,date_sent,subject,email_body
0,james_ngola2002@maktoob.com,MR. JAMES NGOLA.,james_ngola2002@maktoob.com,None,31 Oct 2002,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-2...
1,bensul2004nng@spinfinder.com,Mr. Ben Suleman,R@M,None,31 Oct 2002,URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ..."
2,obong_715@epatra.com,PRINCE OBONG ELEME,obong_715@epatra.com,None,31 Oct 2002,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
3,obong_715@epatra.com,PRINCE OBONG ELEME,webmaster@aclweb.org,None,31 Oct 2002,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
4,m_abacha03@www.com,Maryam Abacha,m_abacha03@www.com,None,1 Nov 2002,I Need Your Assistance.,"Dear sir, \n \nIt is with a heart full of hope..."
5,davidkuta@postmark.net,None,davidkuta@yahoo.com,None,02 Nov 2002,Partnership,ATTENTION: ...
6,tunde_dosumu@lycos.com,Barrister tunde dosumu,tunde_dosumu@lycos.com,None,None,Urgent Attention,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)..."
7,william2244drallo@maktoob.com,William Drallo,william2244drallo@maktoob.com,None,3 Nov 2002,URGENT BUSINESS PRPOSAL,FROM: WILLIAM DRALLO.\nCONFIDENTIAL TEL: 233-2...
8,abdul_817@rediffmail.com,MR USMAN ABDUL,R@M,None,04 Nov 2002,THANK YOU,"CHALLENGE SECURITIES LTD.\nLAGOS, NIGERIA\n\n\..."
9,barrister_td@lycos.com,Tunde Dosumu,barrister_td@lycos.com,None,None,Urgent Assistance,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)..."
